## packages

In [ ]:
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

import numpy as np
import itertools
import pandas as pd
import pandas as pd
import spacy
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.5/149.5 kB 1.4 MB/s eta 0:00:00


## we use model "en_core_web_sm"

In [ ]:
nlp = spacy.load("en_core_web_sm") # you can use other models as well

In [ ]:
TrainData = "/content/drive/MyDrive/Colab Notebooks/Virtual_Assistance_Asg2/train.data"
DevData = "/content/drive/MyDrive/Colab Notebooks/Virtual_Assistance_Asg2/dev.data"
TestData = "/content/drive/MyDrive/Colab Notebooks/Virtual_Assistance_Asg2/test.data"


## Load data

In [ ]:

train_data = pd.read_csv(TrainData, sep = '\t',
                        names = ["Topic_Id", "Topic_Name", "Sent_1", "Sent_2", "Label", "Sent_1_tag", "Sent_2_tag"])
train_data

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag
0,4,1st QB,EJ Manuel the 1st QB to go in this draft,But my bro from the 757 EJ Manuel is the 1st Q...,"(5, 0)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,But/O/CC/O/O my/O/PRP$/B-NP/O bro/O/NN/I-NP/O ...
1,4,1st QB,EJ Manuel the 1st QB to go in this draft,Can believe EJ Manuel went as the 1st QB in th...,"(5, 0)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,Can/O/MD/B-VP/O believe/O/VB/I-VP/B-EVENT EJ/B...
2,4,1st QB,EJ Manuel the 1st QB to go in this draft,EJ MANUEL IS THE 1ST QB what,"(3, 2)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,EJ/B-person/NNP/B-NP/O MANUEL/I-person/NNP/I-N...
3,4,1st QB,EJ Manuel the 1st QB to go in this draft,EJ da 1st QB off da board,"(2, 3)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,EJ/O/NNP/B-NP/O da/O/DT/I-NP/O 1st/O/CD/I-NP/O...
4,4,1st QB,EJ Manuel the 1st QB to go in this draft,Manuel is the 1st QB to get drafted,"(4, 1)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,Manuel/B-person/NNP/B-NP/O is/O/VBZ/B-VP/O the...
...,...,...,...,...,...,...,...
13058,1891,iPhone 5,It fits the larger iPhone 5,My brother thirsty for this iPhone 5 so ill gu...,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,My/O/PRP$/B-NP/O brother/O/NN/I-NP/O thirsty/O...
13059,1891,iPhone 5,It fits the larger iPhone 5,Should I get the iPhone 5 or an Android,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Should/O/UH/O/O I/O/PRP/B-NP/O get/O/VBP/B-VP/...
13060,1891,iPhone 5,It fits the larger iPhone 5,Somebody bring me an iPhone 5 charger to my work,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Somebody/O/NN/B-NP/O bring/O/VB/B-VP/B-EVENT m...
13061,1891,iPhone 5,It fits the larger iPhone 5,Unlocked iPhone 5 300 hit me,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Unlocked/O/NNP/B-NP/O iPhone/O/NNP/I-NP/O 5/O/...


In [ ]:
dev_data = pd.read_csv(DevData, sep = '\t',
                     names = ["Topic_Id", "Topic_Name", "Sent_1", "Sent_2", "Label", "Sent_1_tag", "Sent_2_tag"])
dev_data

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...
...,...,...,...,...,...,...,...
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...


In [ ]:
test_data = pd.read_csv(TestData, sep = '\t', names = ["Topic_Id", "Topic_Name", "Sent_1", "Sent_2", "Label", "Sent_1_tag", "Sent_2_tag"])
test_data

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag
0,51,8 Mile,All the home alones watching 8 mile,8 mile is on thats my movie,3,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP...,8/O/NN/B-NP/O mile/O/NN/I-NP/O is/O/VBZ/B-VP/O...
1,51,8 Mile,All the home alones watching 8 mile,The last rap battle in 8 Mile nevr gets old ahah,2,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP...,The/O/DT/B-NP/O last/O/JJ/I-NP/O rap/O/NN/I-NP...
2,51,8 Mile,All the home alones watching 8 mile,The rap battle at the end of 8 mile gets me so...,2,All/O/DT/B-NP/O the/O/DT/I-NP/O home/O/NN/I-NP...,The/O/DT/B-NP/O rap/O/NN/I-NP/O battle/O/NN/I-...
3,51,8 Mile,The Ending to 8 Mile is my fav part of the who...,Rabbit on 8 mile out of place but determined t...,1,The/O/DT/B-NP/O Ending/O/VBG/I-NP/B-EVENT to/O...,Rabbit/O/NNP/B-NP/O on/O/IN/B-PP/O 8/O/CD/B-NP...
4,51,8 Mile,The Ending to 8 Mile is my fav part of the who...,See 8 Mile is always on but it s the tv versio...,1,The/O/DT/B-NP/O Ending/O/VBG/I-NP/B-EVENT to/O...,See/O/VB/B-VP/O 8/O/CD/B-NP/O Mile/O/NNP/I-NP/...
...,...,...,...,...,...,...,...
967,5273,Z-Bo,Ball dont lie ZBo,If Zbo didnt make that shot,2,Ball/O/NN/B-NP/O dont/O/MD/B-VP/O lie/O/VB/I-V...,If/O/IN/B-SBAR/O Zbo/O/NNP/B-NP/O didnt/O/MD/B...
968,5273,Z-Bo,The Spurs own ZBo,ZBo is a fucking scrub,1,The/B-sportsteam/DT/B-NP/O Spurs/I-sportsteam/...,ZBo/B-person/NNP/B-NP/O is/O/VBZ/B-VP/O a/O/DT...
969,5273,Z-Bo,The Spurs own ZBo,ZBo hit these free throws bruh,1,The/B-sportsteam/DT/B-NP/O Spurs/I-sportsteam/...,ZBo/O/MD/B-VP/O hit/O/VB/I-VP/B-EVENT these/O/...
970,5273,Z-Bo,The Spurs own ZBo,I told you feed Zbo,1,The/B-sportsteam/DT/B-NP/O Spurs/I-sportsteam/...,I/O/PRP/B-NP/O told/O/VBD/B-VP/B-EVENT you/O/P...


## convert label to binary classifiacation

(non-paraphrases) (0, 5), (1, 4), (2, 3) -> 0

(Paraphrases)     (3, 2), (4, 1), (5, 0) -> 1

In [ ]:
#   Preprocessing the data to add binary labels
def convert_labels(data):
  bin_labels = []
  for label in data["Label"]:
      if int(label[1]) > int(label[4]):
          bin_labels.append(1)
      else:
          bin_labels.append(0)

  data["Bin_label"] = bin_labels
  return data

In [ ]:
#  Preprocessing the train data to add binary labels
train_data = convert_labels(train_data)
train_data


,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
0,4,1st QB,EJ Manuel the 1st QB to go in this draft,But my bro from the 757 EJ Manuel is the 1st Q...,"(5, 0)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,But/O/CC/O/O my/O/PRP$/B-NP/O bro/O/NN/I-NP/O ...,1
1,4,1st QB,EJ Manuel the 1st QB to go in this draft,Can believe EJ Manuel went as the 1st QB in th...,"(5, 0)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,Can/O/MD/B-VP/O believe/O/VB/I-VP/B-EVENT EJ/B...,1
2,4,1st QB,EJ Manuel the 1st QB to go in this draft,EJ MANUEL IS THE 1ST QB what,"(3, 2)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,EJ/B-person/NNP/B-NP/O MANUEL/I-person/NNP/I-N...,1
3,4,1st QB,EJ Manuel the 1st QB to go in this draft,EJ da 1st QB off da board,"(2, 3)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,EJ/O/NNP/B-NP/O da/O/DT/I-NP/O 1st/O/CD/I-NP/O...,0
4,4,1st QB,EJ Manuel the 1st QB to go in this draft,Manuel is the 1st QB to get drafted,"(4, 1)",EJ/B-person/NNP/B-NP/O Manuel/I-person/NNP/B-V...,Manuel/B-person/NNP/B-NP/O is/O/VBZ/B-VP/O the...,1
...,...,...,...,...,...,...,...,...
13058,1891,iPhone 5,It fits the larger iPhone 5,My brother thirsty for this iPhone 5 so ill gu...,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,My/O/PRP$/B-NP/O brother/O/NN/I-NP/O thirsty/O...,0
13059,1891,iPhone 5,It fits the larger iPhone 5,Should I get the iPhone 5 or an Android,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Should/O/UH/O/O I/O/PRP/B-NP/O get/O/VBP/B-VP/...,0
13060,1891,iPhone 5,It fits the larger iPhone 5,Somebody bring me an iPhone 5 charger to my work,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Somebody/O/NN/B-NP/O bring/O/VB/B-VP/B-EVENT m...,0
13061,1891,iPhone 5,It fits the larger iPhone 5,Unlocked iPhone 5 300 hit me,"(0, 5)",It/O/PRP/B-NP/O fits/O/VBZ/B-VP/O the/O/DT/B-N...,Unlocked/O/NNP/B-NP/O iPhone/O/NNP/I-NP/O 5/O/...,0


In [ ]:
# Preprocessing the dev data to add binary labels
dev_data = convert_labels(dev_data)
dev_data

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0
...,...,...,...,...,...,...,...,...
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...,0
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...,0


In [ ]:
# show the 0 label data
dev_data[dev_data["Bin_label"]==0]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0
5,17,A Walk To Remember,A Walk to Remember is the definition of true love,Day is made A Walk to Remember is on,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,Day/O/NN/B-NP/O is/O/VBZ/B-VP/O made/O/VBN/I-V...,0
6,17,A Walk To Remember,A Walk to Remember is the definition of true love,I have to watch A Walk to Remember every time ...,"(2, 3)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,I/O/PRP/B-NP/O have/O/VBP/B-VP/O to/O/TO/I-VP/...,0
...,...,...,...,...,...,...,...,...
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...,0
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...,0


In [ ]:
import random
randomRow = np.random.randint(4726, size=1)
print(dev_data.iloc[randomRow[0], :])

Topic_Id                                                    836
Topic_Name                                                 Judi
Sent_1                           Judi was psycho lmao loved her
Sent_2        LMFAO YOOO when Judi threw cereal on Priscilla...
Label                                                    (3, 2)
Sent_1_tag    Judi/B-band/NNP/B-NP/O was/O/VBD/B-VP/O psycho...
Sent_2_tag    LMFAO/O/UH/B-INTJ/O YOOO/O/UH/O/O when/O/WRB/B...
Bin_label                                                     1
Name: 2473, dtype: object


In [ ]:
# show the 1 label data
dev_data[dev_data["Bin_label"]==1]


,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
17,18,A Walk to Remember,On the real I like the movie A Walk to Remember,Watching A Walk To Remember is seriously makin...,"(3, 2)",On/O/IN/B-PP/O the/O/DT/B-NP/O real/O/JJ/I-NP/...,Watching/O/VBG/B-VP/B-EVENT A/O/DT/B-NP/O Walk...,1
20,38,Adidas,AAP is in the Adidas commercial,AAP in that Adidas Commercial lol,"(4, 1)",AAP/B-person/NNP/B-NP/O is/O/VBZ/B-VP/B-EVENT ...,AAP/O/NNP/B-NP/O in/O/IN/B-PP/O that/O/DT/B-NP...,1
21,38,Adidas,AAP is in the Adidas commercial,AAP was nice on that Adidas commercial,"(4, 1)",AAP/B-person/NNP/B-NP/O is/O/VBZ/B-VP/B-EVENT ...,AAP/B-person/NNP/B-NP/O was/O/VBD/B-VP/O nice/...,1
...,...,...,...,...,...,...,...,...
4664,1847,Z-Bo,ZBo is torching the clippers,ZBo rules the world bitch,"(4, 1)",ZBo/O/NNP/B-NP/O is/O/VBZ/B-VP/O torching/O/VB...,ZBo/O/JJ/B-NP/O rules/O/NNS/I-NP/O the/O/DT/B-...,1
4666,1847,Z-Bo,ZBo is torching the clippers,lol they getting abuse by Gasol and ZBo,"(3, 2)",ZBo/O/NNP/B-NP/O is/O/VBZ/B-VP/O torching/O/VB...,lol/O/UH/B-INTJ/O they/O/PRP/B-NP/O getting/O/...,1
4709,1890,iPhone 4,Lmfaaaaooooo she said they shaped like a iPhone 4,Hahahahahah She Said They Shaped Like iPhone 4 s,"(4, 1)",Lmfaaaaooooo/O/IN/B-SBAR/O she/O/PRP/B-NP/O sa...,Hahahahahah/O/IN/B-SBAR/O She/O/PRP/B-NP/O Sai...,1
4711,1890,iPhone 4,Lmfaaaaooooo she said they shaped like a iPhone 4,She said the twins shaped like iPhone 4 lmfaoo...,"(3, 2)",Lmfaaaaooooo/O/IN/B-SBAR/O she/O/PRP/B-NP/O sa...,She/O/PRP/B-NP/O said/O/VBD/B-VP/B-EVENT the/O...,1


In [ ]:
# The test data to add binary labels
test_data.Label.value_counts()

1    376
0    157
3    134
4    134
2    130
5     41
Name: Label, dtype: int64

In [ ]:
def convert_label_test(data):

  # Change labels: 3 -> drop
  data = data.drop(data[data.Label == 3].index)

  # Change labels: 0, 1, 2 -> 0
  # Change labels: 4, 5 -> 1
  data.Label = data.Label.replace({0: 0, 1: 0, 2: 0, 4: 1, 5: 1})

  return data

In [ ]:
# Distribution the test data to add binary labels
test_data = convert_label_test(test_data)
test_data.Label.value_counts()

0    663
1    175
Name: Label, dtype: int64

In [ ]:
# dev data 1 label
dev_data[dev_data["Bin_label"] == 1]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
17,18,A Walk to Remember,On the real I like the movie A Walk to Remember,Watching A Walk To Remember is seriously makin...,"(3, 2)",On/O/IN/B-PP/O the/O/DT/B-NP/O real/O/JJ/I-NP/...,Watching/O/VBG/B-VP/B-EVENT A/O/DT/B-NP/O Walk...,1
20,38,Adidas,AAP is in the Adidas commercial,AAP in that Adidas Commercial lol,"(4, 1)",AAP/B-person/NNP/B-NP/O is/O/VBZ/B-VP/B-EVENT ...,AAP/O/NNP/B-NP/O in/O/IN/B-PP/O that/O/DT/B-NP...,1
21,38,Adidas,AAP is in the Adidas commercial,AAP was nice on that Adidas commercial,"(4, 1)",AAP/B-person/NNP/B-NP/O is/O/VBZ/B-VP/B-EVENT ...,AAP/B-person/NNP/B-NP/O was/O/VBD/B-VP/O nice/...,1
...,...,...,...,...,...,...,...,...
4664,1847,Z-Bo,ZBo is torching the clippers,ZBo rules the world bitch,"(4, 1)",ZBo/O/NNP/B-NP/O is/O/VBZ/B-VP/O torching/O/VB...,ZBo/O/JJ/B-NP/O rules/O/NNS/I-NP/O the/O/DT/B-...,1
4666,1847,Z-Bo,ZBo is torching the clippers,lol they getting abuse by Gasol and ZBo,"(3, 2)",ZBo/O/NNP/B-NP/O is/O/VBZ/B-VP/O torching/O/VB...,lol/O/UH/B-INTJ/O they/O/PRP/B-NP/O getting/O/...,1
4709,1890,iPhone 4,Lmfaaaaooooo she said they shaped like a iPhone 4,Hahahahahah She Said They Shaped Like iPhone 4 s,"(4, 1)",Lmfaaaaooooo/O/IN/B-SBAR/O she/O/PRP/B-NP/O sa...,Hahahahahah/O/IN/B-SBAR/O She/O/PRP/B-NP/O Sai...,1
4711,1890,iPhone 4,Lmfaaaaooooo she said they shaped like a iPhone 4,She said the twins shaped like iPhone 4 lmfaoo...,"(3, 2)",Lmfaaaaooooo/O/IN/B-SBAR/O she/O/PRP/B-NP/O sa...,She/O/PRP/B-NP/O said/O/VBD/B-VP/B-EVENT the/O...,1


In [ ]:
dev_data.head()

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0


In [ ]:
# dev data 0 label

predict_algo_a = []
for i in range(len(dev_data)):
    if dev_data.iloc[i][2] == dev_data.iloc[i][3]:
        predict_algo_a.append(1)
    else:
        predict_algo_a.append(0)

dev_data["Predict_AlgoA"] = predict_algo_a
dev_data.head()


,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0,0


In [ ]:
dev_data["Predict_AlgoA"].value_counts()

0    4724
1       3
Name: Predict_AlgoA, dtype: int64

In [ ]:
# dev data 0 label
dev_data[dev_data["Predict_AlgoA"]==0]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0,0
...,...,...,...,...,...,...,...,...,...
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...,0,0
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0,0
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0,0
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...,0,0


In [ ]:
import random
randomRow = np.random.randint(4726, size=1)
print(dev_data.iloc[randomRow[0], :])

Topic_Id                                                      1781
Topic_Name                                                    Vick
Sent_1                                  Vick crushed McCoy in a 40
Sent_2                     Mike Vick tore up Shady in the 40 today
Label                                                       (4, 1)
Sent_1_tag       Vick/B-geo-loc/NNP/B-NP/O crushed/O/VBD/B-VP/B...
Sent_2_tag       Mike/B-person/NNP/B-NP/O Vick/I-person/NNP/I-N...
Bin_label                                                        1
Predict_AlgoA                                                    0
Name: 4562, dtype: object


In [ ]:
# example of 1 label data
dev_data[dev_data["Predict_AlgoA"]==1]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA
50,72,Amber,Do not Amber Alert me,Do not Amber Alert me,"(5, 0)",Do/O/VBP/B-VP/O not/O/RB/I-VP/O Amber/O/RB/I-V...,Do/O/VBP/B-VP/O not/O/RB/I-VP/O Amber/O/RB/I-V...,1,1
1274,322,Coach Carter,Need to do homework but Coach Carter is on,Need to do homework but Coach Carter is on,"(5, 0)",Need/O/VB/B-VP/O to/O/TO/I-VP/O do/O/VB/I-VP/O...,Need/O/VB/B-VP/O to/O/TO/I-VP/O do/O/VB/I-VP/O...,1,1
4383,1771,Vedo,Vedo I love this guy,Vedo I love this guy,"(5, 0)",Vedo/O/NN/B-NP/O I/O/PRP/B-NP/O love/O/VBP/B-V...,Vedo/O/NN/B-NP/O I/O/PRP/B-NP/O love/O/VBP/B-V...,1,1


In [ ]:
print(dev_data.iloc[50, :])

Topic_Id                                                        72
Topic_Name                                                   Amber
Sent_1                                       Do not Amber Alert me
Sent_2                                       Do not Amber Alert me
Label                                                       (5, 0)
Sent_1_tag       Do/O/VBP/B-VP/O not/O/RB/I-VP/O Amber/O/RB/I-V...
Sent_2_tag       Do/O/VBP/B-VP/O not/O/RB/I-VP/O Amber/O/RB/I-V...
Bin_label                                                        1
Predict_AlgoA                                                    1
Name: 50, dtype: object


In [ ]:
# load AlgoA confusion matrix
AlgoA_cm = confusion_matrix(dev_data["Bin_label"], dev_data["Predict_AlgoA"])
AlgoA_cm

array([[3257,    0],
       [1467,    3]])

In [ ]:
# Precision of AlgoA
AlgoA_precision = precision_score(dev_data["Bin_label"], dev_data["Predict_AlgoA"])
AlgoA_precision

1.0

In [ ]:
# Recall of AlgoA
AlgoA_recall = recall_score(dev_data["Bin_label"], dev_data["Predict_AlgoA"])
AlgoA_recall

0.0020408163265306124

In [ ]:
# F1 score of AlgoA
AlgoA_f1score = f1_score(dev_data["Bin_label"], dev_data["Predict_AlgoA"])
AlgoA_f1score

0.004073319755600815

## Algorithm B

In [ ]:
len(dev_data)

4727

In [ ]:
# Number of all sentences
all_sentences=[]
for i in range (len(dev_data)):
  all_sentences.append(dev_data.iloc[i]["Sent_1"])
  all_sentences.append(dev_data.iloc[i]["Sent_2"])

#Get unique sentences
all_sentences = list(set(all_sentences))

print(f"Number of all sentences: {len(all_sentences)}")

Number of all sentences: 4772


In [ ]:
# convert to spacy vectors
list_allSentences = [nlp(sentence).vector for sentence in all_sentences]

# convert to numpy array
list_allSentences = np.array(list_allSentences)

In [ ]:
X = pd.DataFrame(list_allSentences)
X

,0,1,2,3,4,5,6,7,8,9,...,86,87,88,89,90,91,92,93,94,95
0,0.096285,-0.216692,-0.035472,-0.023766,-0.105508,-0.376996,0.598308,0.182322,0.310640,-0.162092,...,0.097215,-0.194651,0.163431,0.645278,0.576155,-0.410603,0.791875,0.742869,-0.793516,0.450194
1,-0.095765,-0.334734,-0.107992,0.065887,-0.219274,-0.208011,0.377598,-0.223052,-0.029847,0.528503,...,0.325335,-0.389628,0.197107,0.181858,-0.223331,0.168710,0.727741,0.210894,-0.436119,0.421416
2,0.128909,0.032816,-0.056114,0.408525,0.040211,-0.153059,0.225240,0.022647,0.301602,0.208528,...,0.447998,0.049707,-0.315426,0.319586,-0.440277,0.181804,0.931328,0.411459,-0.393989,-0.015885
3,-0.024797,-0.616001,0.178433,0.129275,-0.011017,-0.369420,0.288689,-0.248952,-0.118138,0.429333,...,0.018994,-0.114177,0.085307,0.237759,0.276262,-0.184136,0.292446,0.416956,-0.565414,0.072730
4,-0.607797,-0.721087,-0.098006,0.226674,-0.309610,-0.088157,0.047575,0.501112,0.081683,0.336768,...,0.604287,-0.171909,-0.430970,0.629721,-0.513414,-0.279215,0.598081,0.272643,-0.387723,0.018867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4767,-0.129628,-0.304531,0.084847,-0.155617,0.205805,-0.355742,0.394564,0.464721,0.223270,0.392580,...,0.141597,-0.222691,-0.554658,0.549215,-0.026521,0.211292,0.894523,0.310129,-0.252938,0.215838
4768,-0.361349,-0.456786,-0.065726,0.744785,-0.145193,-0.116555,0.587290,0.201561,0.017428,-0.068635,...,-0.073021,-0.557558,0.230239,-0.295552,0.095157,-0.137688,1.176268,0.225448,-0.284635,0.829253
4769,0.034388,-0.352034,0.067779,0.250565,-0.101846,-0.045710,0.334020,0.397404,-0.070938,0.471158,...,0.499893,-0.001630,-0.568055,0.219336,-0.258571,-0.288425,0.865807,0.343887,-0.646145,-0.116035
4770,-0.602070,-0.400314,-0.206434,0.288571,0.502028,-0.295121,0.224268,0.675050,0.182730,-0.063881,...,0.249379,0.013370,-0.738744,0.779695,0.274444,-0.413005,1.032139,0.539206,-0.017822,0.178825


In [ ]:
# Kmeans clustering
kmeans_pk = []
numberOfClusters = [3, 5, 7, 9, 11, 13]
for k in numberOfClusters:
    kmeans_model = KMeans(n_clusters=k, random_state=0).fit(X)
    kmeans_pk.append(kmeans_model)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
# Silhouette scores
silhouette_scores = []
for model in kmeans_pk:
    score = silhouette_score(X, model.labels_)
    silhouette_scores.append((model, score))

silhouette_scores

[(KMeans(n_clusters=3, random_state=0), 0.057175826),
 (KMeans(n_clusters=5, random_state=0), 0.052153025),
 (KMeans(n_clusters=7, random_state=0), 0.04379582),
 (KMeans(n_clusters=9, random_state=0), 0.04064872),
 (KMeans(n_clusters=11, random_state=0), 0.03948293),
 (KMeans(n_clusters=13, random_state=0), 0.032690696)]

In [ ]:
# The best model
model_B = kmeans_pk[0]
model_B

KMeans(n_clusters=3, random_state=0)

In [ ]:
# Predict the clusters
# every sentence is assigned to a cluster based on the closest centroid
# the cluster number is the label of the sentence
predictions = []
for i in range(len(dev_data)):
    sent_1_vector = nlp(dev_data.iloc[i]["Sent_1"]).vector
    sent_2_vector = nlp(dev_data.iloc[i]["Sent_2"]).vector
    prediction = model_B.predict(np.array([sent_1_vector, sent_2_vector]))
    predictions.append(prediction)

predict_algo_b = []
for i in predictions:
    if i[0] == i[1]:
        predict_algo_b.append(1)
    else:
        predict_algo_b.append(0)

dev_data["Predict_AlgoB"] = predict_algo_b


In [ ]:
#Examples of Algo B 1 label data
dev_data[dev_data["Predict_AlgoB"]==1]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA,Predict_AlgoB
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0,1
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0,0,1
9,17,A Walk To Remember,A Walk to Remember is the definition of true love,Watching A Walk To Remember for the millionth ...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,Watching/O/VBG/B-VP/B-EVENT A/O/DT/B-NP/O Walk...,0,0,1
12,18,A Walk to Remember,On the real I like the movie A Walk to Remember,I want a love like Jamie and Landon on A Walk ...,"(0, 5)",On/O/IN/B-PP/O the/O/DT/B-NP/O real/O/JJ/I-NP/...,I/O/PRP/B-NP/O want/O/VBP/B-VP/O a/O/DT/B-NP/O...,0,0,1
14,18,A Walk to Remember,On the real I like the movie A Walk to Remember,THE GUY IN A WALK TO REMEMBER IS SO CUTE IM PI...,"(1, 4)",On/O/IN/B-PP/O the/O/DT/B-NP/O real/O/JJ/I-NP/...,THE/O/DT/B-NP/O GUY/O/NN/I-NP/O IN/O/IN/B-PP/O...,0,0,1
...,...,...,...,...,...,...,...,...,...,...
4721,1902,iTunes,My first podcast drops on iTunes,Limited time only get Opposites for 999 on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Limited/O/JJ/B-NP/O time/O/NN/I-NP/O only/O/RB...,0,0,1
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0,0,1
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0,0,1
4725,1902,iTunes,My first podcast drops on iTunes,iTunes deleted my voice memos,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,iTunes/B-company/NNP/B-NP/O deleted/O/VBD/B-VP...,0,0,1


In [ ]:
#Examples of Algo B 0 label data
dev_data[dev_data["Predict_AlgoB"]==0]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA,Predict_AlgoB
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0,0
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0,0
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0,0
5,17,A Walk To Remember,A Walk to Remember is the definition of true love,Day is made A Walk to Remember is on,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,Day/O/NN/B-NP/O is/O/VBZ/B-VP/O made/O/VBN/I-V...,0,0,0
6,17,A Walk To Remember,A Walk to Remember is the definition of true love,I have to watch A Walk to Remember every time ...,"(2, 3)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,I/O/PRP/B-NP/O have/O/VBP/B-VP/O to/O/TO/I-VP/...,0,0,0
...,...,...,...,...,...,...,...,...,...,...
4714,1890,iPhone 4,Lmfaaaaooooo she said they shaped like a iPhone 4,This damn girl said she shape like a iphone 4,"(1, 4)",Lmfaaaaooooo/O/IN/B-SBAR/O she/O/PRP/B-NP/O sa...,This/O/DT/B-NP/O damn/O/JJ/I-NP/O girl/O/NN/I-...,0,0,0
4717,1902,iTunes,My first podcast drops on iTunes,And i can control my Mac itunes using my iPhone,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,And/O/CC/O/O i/O/PRP/B-NP/O can/O/MD/B-VP/O co...,0,0,0
4719,1902,iTunes,My first podcast drops on iTunes,GET IT TO NUMBER ONE ON ITUNES,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,GET/O/VB/B-VP/O IT/O/PRP/B-NP/O TO/O/TO/B-VP/O...,0,0,0
4720,1902,iTunes,My first podcast drops on iTunes,LETS MAKE IT NUMBER 1 ON ITUNES,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,LETS/O/NNS/B-NP/O MAKE/O/VBP/B-VP/O IT/O/PRP/B...,0,0,0


In [ ]:
# AlgoB confusion matrix
AlgoB_cm = confusion_matrix(dev_data["Bin_label"], dev_data["Predict_AlgoB"])
AlgoB_cm

array([[1905, 1352],
       [ 703,  767]])

In [ ]:
# Precision of AlgoB
AlgoB_precision = precision_score(dev_data["Bin_label"], dev_data["Predict_AlgoB"])
AlgoB_precision

0.3619631901840491

In [ ]:
# Recall of AlgoB
AlgoB_recall = recall_score(dev_data["Bin_label"], dev_data["Predict_AlgoB"])
AlgoB_recall

0.5217687074829932

In [ ]:
# f1 score of AlgoB
AlgoB_f1score = f1_score(dev_data["Bin_label"], dev_data["Predict_AlgoB"])
AlgoB_f1score

0.427417107829479

# Algorithm C - kmeans with BERT

In [ ]:
# Load roberta model
from sentence_transformers import SentenceTransformer
robert_model = SentenceTransformer('stsb-roberta-large')
# sbert_model = SentenceTransformer('bert-base-nli-mean-tokens')

In [ ]:
# Get the sentence embeddings
sentence_embeddings = robert_model.encode(all_sentences)

In [ ]:
# Get the shape of the sentence embeddings
sentence_embeddings.shape

(4772, 1024)

In [ ]:
# Kmeans clustering
kmeans_per_k = []
numberOfClusters = [3, 5, 7, 9, 11, 13, 15, 17, 19 ,21, 23, 25]
for k in numberOfClusters:
    kmeans_model = KMeans(n_clusters=k, random_state=0).fit(sentence_embeddings)
    kmeans_per_k.append(kmeans_model)


/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [ ]:
# Silhouette scores
silhouette_scores = []
for model in kmeans_per_k:
    score = silhouette_score(sentence_embeddings, model.labels_)
    silhouette_scores.append((model, score))

silhouette_scores

[(KMeans(n_clusters=3, random_state=0), 0.028906113),
 (KMeans(n_clusters=5, random_state=0), 0.036998987),
 (KMeans(n_clusters=7, random_state=0), 0.04007484),
 (KMeans(n_clusters=9, random_state=0), 0.046507176),
 (KMeans(n_clusters=11, random_state=0), 0.05377753),
 (KMeans(n_clusters=13, random_state=0), 0.057790056),
 (KMeans(n_clusters=15, random_state=0), 0.06198403),
 (KMeans(n_clusters=17, random_state=0), 0.06371885),
 (KMeans(n_clusters=19, random_state=0), 0.07084537),
 (KMeans(n_clusters=21, random_state=0), 0.07075259),
 (KMeans(n_clusters=23, random_state=0), 0.077532865),
 (KMeans(n_clusters=25, random_state=0), 0.08241536)]

In [ ]:
# The best model
k_means_selection = kmeans_per_k[11]
k_means_selection

KMeans(n_clusters=25, random_state=0)

In [ ]:
# cluster the sentences
sent_1=[]
sent_2=[]
for i in range (len(dev_data)):
  sent_1.append(dev_data.iloc[i]["Sent_1"])
  sent_2.append(dev_data.iloc[i]["Sent_2"])

sent1_embeddings = robert_model.encode(sent_1)
sent2_embeddings = robert_model.encode(sent_2)

In [ ]:
# shape of the sentence embeddings
sent1_embeddings.shape

(4727, 1024)

In [ ]:
# reshape the sentence embeddings
s1 = sent1_embeddings.reshape(4727,1024)
s2 = sent2_embeddings.reshape(4727,1024)

In [ ]:
# Predict the clusters
# every sentence is assigned to a cluster based on the closest centroid
predictions=[]
for i in range (len(dev_data)):
  cluster1 =k_means_selection.predict(sent1_embeddings[i].reshape(1,1024))
  cluster2 =k_means_selection.predict(sent2_embeddings[i].reshape(1,1024))

  if cluster1 ==cluster2:
    predictions.append(1)
  else:
    predictions.append(0)

In [ ]:
# add the predictions to the dev data
dev_data["Predict_AlgoC"] = predictions

In [ ]:
#Examples of Algo B
dev_data[dev_data["Predict_AlgoC"]==0]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA,Predict_AlgoB,Predict_AlgoC
0,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is on and Im in town and Im...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0,0,0
2,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is on ABC family youre welcome,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,0,0,0,0
5,17,A Walk To Remember,A Walk to Remember is the definition of true love,Day is made A Walk to Remember is on,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,Day/O/NN/B-NP/O is/O/VBZ/B-VP/O made/O/VBN/I-V...,0,0,0,0
6,17,A Walk To Remember,A Walk to Remember is the definition of true love,I have to watch A Walk to Remember every time ...,"(2, 3)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,I/O/PRP/B-NP/O have/O/VBP/B-VP/O to/O/TO/I-VP/...,0,0,0,0
7,17,A Walk To Remember,A Walk to Remember is the definition of true love,Oh god a walk to remember is on bring on the w...,"(2, 3)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,Oh/O/UH/B-INTJ/O god/O/UH/I-INTJ/O a/O/DT/B-NP...,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
4718,1902,iTunes,My first podcast drops on iTunes,Btw my single Waiting For Your Love is availab...,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Btw/O/UH/B-INTJ/O my/O/PRP$/B-NP/O single/O/NN...,0,0,1,0
4719,1902,iTunes,My first podcast drops on iTunes,GET IT TO NUMBER ONE ON ITUNES,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,GET/O/VB/B-VP/O IT/O/PRP/B-NP/O TO/O/TO/B-VP/O...,0,0,0,0
4720,1902,iTunes,My first podcast drops on iTunes,LETS MAKE IT NUMBER 1 ON ITUNES,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,LETS/O/NNS/B-NP/O MAKE/O/VBP/B-VP/O IT/O/PRP/B...,0,0,0,0
4721,1902,iTunes,My first podcast drops on iTunes,Limited time only get Opposites for 999 on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Limited/O/JJ/B-NP/O time/O/NN/I-NP/O only/O/RB...,0,0,1,0


In [ ]:
randomRow = np.random.randint(4726, size=1)
print(dev_data.iloc[randomRow[0], :])

Topic_Id                                                      1568
Topic_Name                                              Summer Jam
Sent_1                          Wow Summer Jam is going to be live
Sent_2                            Yo who wants to go to summer jam
Label                                                       (0, 5)
Sent_1_tag       Wow/B-band/NNP/B-NP/O Summer/I-band/NNP/I-NP/O...
Sent_2_tag       Yo/O/NNS/B-NP/O who/O/WP/B-NP/O wants/O/VBZ/B-...
Bin_label                                                        0
Predict_AlgoA                                                    0
Predict_AlgoB                                                    0
Predict_AlgoC                                                    0
Name: 4103, dtype: object


In [ ]:
#Examples of Algo B
dev_data[dev_data["Predict_AlgoC"]==1]

,Topic_Id,Topic_Name,Sent_1,Sent_2,Label,Sent_1_tag,Sent_2_tag,Bin_label,Predict_AlgoA,Predict_AlgoB,Predict_AlgoC
1,17,A Walk To Remember,A Walk to Remember is the definition of true love,A Walk to Remember is the cutest thing,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0,1,1
3,17,A Walk To Remember,A Walk to Remember is the definition of true love,A walk to remember is so amazing and inspiring,"(3, 2)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,A/O/DT/B-NP/O walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,1,0,0,1
4,17,A Walk To Remember,A Walk to Remember is the definition of true love,BUT GUYS ITS ON MY FAVE PART OF A WALK TO REME...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,BUT/O/CC/O/O GUYS/O/VBP/B-VP/B-EVENT ITS/O/PRP...,0,0,1,1
8,17,A Walk To Remember,A Walk to Remember is the definition of true love,The only Nicholas Sparks movie I genuinely lik...,"(1, 4)",A/O/DT/B-NP/O Walk/O/NN/I-NP/O to/O/TO/B-VP/O ...,The/O/DT/B-NP/O only/O/JJ/I-NP/O Nicholas/B-pe...,0,0,0,1
14,18,A Walk to Remember,On the real I like the movie A Walk to Remember,THE GUY IN A WALK TO REMEMBER IS SO CUTE IM PI...,"(1, 4)",On/O/IN/B-PP/O the/O/DT/B-NP/O real/O/JJ/I-NP/...,THE/O/DT/B-NP/O GUY/O/NN/I-NP/O IN/O/IN/B-PP/O...,0,0,1,1
...,...,...,...,...,...,...,...,...,...,...,...
4717,1902,iTunes,My first podcast drops on iTunes,And i can control my Mac itunes using my iPhone,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,And/O/CC/O/O i/O/PRP/B-NP/O can/O/MD/B-VP/O co...,0,0,0,1
4722,1902,iTunes,My first podcast drops on iTunes,No wonder I spend so much on iTunes,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,No/O/DT/B-NP/O wonder/O/NN/I-NP/O I/O/PRP/B-NP...,0,0,0,1
4723,1902,iTunes,My first podcast drops on iTunes,Our Dead and Gone EP is now on iTunes,"(1, 4)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,Our/O/PRP/B-NP/O Dead/O/JJ/I-NP/O and/O/CC/I-N...,0,0,1,1
4724,1902,iTunes,My first podcast drops on iTunes,amslingshots is 7 on the iTunes charts,"(0, 5)",My/O/PRP$/B-NP/O first/O/JJ/I-NP/O podcast/O/N...,amslingshots/O/NNP/B-NP/O is/O/VBZ/B-VP/O 7/O/...,0,0,1,1


In [ ]:
# number of data
len(predictions)

4727

In [ ]:
# Confusion Matrix
AlgoC_cm = confusion_matrix(dev_data["Bin_label"], dev_data["Predict_AlgoC"])
AlgoC_cm

array([[1314, 1943],
       [ 243, 1227]])

In [ ]:
# precision of AlgoC
AlgoC_precision = precision_score(dev_data["Bin_label"], dev_data["Predict_AlgoC"])
AlgoC_precision

0.38706624605678236

In [ ]:
# Recall of AlgoC
AlgoC_recall = recall_score(dev_data["Bin_label"], dev_data["Predict_AlgoC"])
AlgoC_recall

0.8346938775510204

In [ ]:
# f1 score of AlgoC
AlgoC_f1score = f1_score(dev_data["Bin_label"], dev_data["Predict_AlgoC"])
AlgoC_f1score

0.5288793103448276

In [ ]:
# number of data
len(test_data)

838

In [ ]:
#convert dev sentences to their embeddings
Test_1=[]
Test_2=[]
for i in range (len(test_data)):
  Test_1.append(test_data.iloc[i]["Sent_1"])
  Test_2.append(test_data.iloc[i]["Sent_2"])

Test1_embeddings = robert_model.encode(Test_1)
Test2_embeddings = robert_model.encode(Test_2)

In [ ]:
# shape of the sentence embeddings
Test1_embeddings.shape

(838, 1024)

In [ ]:
# reshape the sentence embeddings
s1_T = Test1_embeddings.reshape(838,1024)
s2_T = Test2_embeddings.reshape(838,1024)

In [ ]:
# Predict the clusters
# every sentence is assigned to a cluster based on the closest centroid
predictions_Test=[]
for i in range (len(test_data)):
  predict1 =k_means_selection.predict(Test1_embeddings[i].reshape(1,1024))
  predict2 =k_means_selection.predict(Test2_embeddings[i].reshape(1,1024))
  if predict1 ==predict2:
    predictions_Test.append(1)
  else:
    predictions_Test.append(0)

In [ ]:
test_data["chosen_method"] = predictions_Test

In [ ]:
# Confusion Matrix for test dataset
confusion_matrix = confusion_matrix(test_data["Label"], test_data["chosen_method"])
confusion_matrix

array([[434, 229],
       [ 78,  97]])

In [ ]:
# Precision for test dataset
precision = precision_score(test_data["Label"], test_data["chosen_method"])
precision

0.29754601226993865

In [ ]:
# Recall for test dataset
recall = recall_score(test_data["Label"], test_data["chosen_method"])
recall

0.5542857142857143

In [ ]:
# f1 score for test dataset
f1score = f1_score(test_data["Label"], test_data["chosen_method"])
f1score

0.38722554890219557